# Tutorial 4: Linear Operators, Duals, and Adjoints

In previous tutorials, we introduced the `HilbertSpace` and the concept of a `dual` space. Now, we'll focus on the `LinearOperator` class, which defines linear maps between these spaces.

### The Theory in Outline

Given a linear operator $A: X \to Y$, we can define two related operators:

1.  **The Dual Operator ($A'$)**: The dual (or transpose) operator maps between the dual spaces, $A': Y' \to X'$. It is defined by the relation:
    $$
    \langle y', A(x) \rangle = \langle A'(y'), x \rangle
    $$
    This relation holds for any $x \in X$ and $y' \in Y'$.

2.  **The Adjoint Operator ($A^*$)**: The adjoint operator maps between the original primal spaces, but in reverse, $A^*: Y \to X$. It is defined by the inner product:
    $$
    \langle y, A(x) \rangle_Y = \langle A^*(y), x \rangle_X
    $$

A key feature of this library is that if you provide just the forward mapping for `A`, the library can automatically derive its `dual` and `adjoint` operators.

In this tutorial, we will:
1.  Create a `LinearOperator` from a simple mapping.
2.  Combine operators using standard arithmetic operations.
3.  Numerically verify the defining relations for the dual and adjoint.
4.  Create an operator on a `Sobolev` space using `from_formal_adjoint`.

In [ ]:
# To run in colab, uncomment the line below to install pygeoinf. 
#%pip install pygeoinf --quiet

import numpy as np
import pygeoinf as inf
import matplotlib.pyplot as plt
from pygeoinf.symmetric_space.circle import Lebesgue, Sobolev
from pygeoinf import EuclideanSpace

# For reproducibility
np.random.seed(123)

## 1. Defining a `LinearOperator`

The most direct way to create a `LinearOperator` is from its constituent parts: a `domain` space, a `codomain` space, and a `mapping` function that defines its action. To illustrate this, lets start with a simple example between Euclidean spaces, namely the projection from $\mathbb{R}^{m}$ to $\mathbb{R}^{n}$ where $m > n$. 


In [ ]:
# Set up the domain and codomain
domain = inf.EuclideanSpace(4)
codomain = inf.EuclideanSpace(2)

# Set up the linear operator
A = inf.LinearOperator(domain, codomain, lambda x : x[:codomain.dim])

# We can test the operator by acting it on a random vector 
x = domain.random()
print(f'Input vector  {x}')
y = A(x)
print(f'Output vector {y}')

As a second, and more elaborate, example we consider the linear operator
$$
(Au)(x) =  a(x) u(x), 
$$
for functions on $L^{2}(\mathbb{S}^{1})$ with $a$ a given smooth function. 

In [ ]:
# Set the domain and codomain (in this case equal)
domain = Lebesgue(256)

# Set the smooth scaling function as an element of the space. 
a = domain.project_function(lambda th : np.exp(-(th-np.pi)**2))

# Set up the linear operator
A = inf.LinearOperator(domain, domain, lambda u : a*u)

# Define a input vector 
u = domain.project_function(lambda th : np.cos(10*th))

# Get the result of acting the operator
v = A(u)

# Plot the input and outputs
fig, ax = domain.plot(u, label='input')
domain.plot(v, fig=fig, ax=ax, label='output')
ax.legend()
ax.set_title("Operator Test")
plt.show()


A `LinearOperator` that has been constructed may not have been done so consistently. For example, the mapping may not be linear. To address this point, a `check` method is available that runs a series of tests on randomised inputs. This is shown below using the operator defined in the previous code-block:

In [ ]:
A.check()

Within the output you will see reference to "non-linear operator checks". This is because  the `LinearOperator` class inherits from a more general `NonLinearOperator` class that will be discussed in a later tutorial (currently to be written).

## 2. Operator Arithmetic

`LinearOperator` objects support standard arithmetic operations, allowing you to build complex operators from simpler ones. You can use:
* `+` and `-` for addition and subtraction.
* `*` for scalar multiplication.
* `@` for composition (matrix multiplication).

In [ ]:
# Define two simple operators on EuclideanSpace
space = EuclideanSpace(2)
M1 = np.array([[1, 2], [3, 4]])
M2 = np.array([[0, 1], [1, 0]])

A = inf.LinearOperator.from_matrix(space, space, M1)
B = inf.LinearOperator.from_matrix(space, space, M2)

# --- Demonstrate arithmetic ---

# Addition: C = A + B
C = A + B
print(f"Matrix of C = A + B:\n{C.matrix(dense=True)}\n")

# Scalar Multiplication: D = 3 * A
D = 3 * A
print(f"Matrix of D = 3 * A:\n{D.matrix(dense=True)}\n")

# Composition: E = A @ B
E = A @ B
print(f"Matrix of E = A @ B:\n{E.matrix(dense=True)}\n")

# We can verify the composition result
assert np.allclose(E.matrix(dense=True), M1 @ M2)

## 3. Duals and Adjoints

The concepts of dual and adjoint operators are easiest to understand in a simple `EuclideanSpace`.

### Providing Mappings vs. Automatic Derivation

When creating a `LinearOperator`, you can optionally provide the `dual_mapping` and `adjoint_mapping` directly in the constructor. This is highly recommended if you know them, as it is much more efficient.

If you **do not** provide them, the library will implement them for you. The default method, however, can be very slow as it involves repeated applications
of the operator. This is fine, if wasteful, for operators on low-dimensional spaces, but for operators on high-dimensional spaces it is not a reasonable option. 

Let's create an operator and verify the defining relations.

In [ ]:
# Define domain and codomain
domain = EuclideanSpace(3)
codomain = EuclideanSpace(2)

# Create an operator from a matrix
M = np.array([[1, 2, 3], [4, 5, 6]])
# We know the adjoint is the transpose, so we can provide it for efficiency
adjoint_map = lambda y: M.T @ y
A = inf.LinearOperator(domain, codomain, lambda x: M @ x, adjoint_mapping=adjoint_map)

# Get the dual and adjoint operators
A_dual = A.dual
A_adjoint = A.adjoint

# Create random vectors for testing
x = domain.random()
y = codomain.random()

# --- Verification ---

# 1. Dual Relation: <y', A(x)> = <A'(y'), x>
# For Euclidean space, the dual of y is a LinearForm with the same components
y_prime = codomain.to_dual(y) 
lhs_dual = codomain.duality_product(y_prime, A(x))
rhs_dual = domain.duality_product(A_dual(y_prime), x)

print(f"--- Dual Operator Verification ---")
print(f"<y', A(x)> = {lhs_dual:.4f}")
print(f"<A'(y'), x> = {rhs_dual:.4f}")
assert np.isclose(lhs_dual, rhs_dual)

# 2. Adjoint Relation: <y, A(x)>_Y = <A*(y), x>_X
lhs_adjoint = codomain.inner_product(y, A(x))
rhs_adjoint = domain.inner_product(A_adjoint(y), x)

print(f"\n--- Adjoint Operator Verification ---")
print(f"<y, A(x)>_Y = {lhs_adjoint:.4f}")
print(f"<A*(y), x>_X = {rhs_adjoint:.4f}")
assert np.isclose(lhs_adjoint, rhs_adjoint)

## 4. Adjoints in Function Spaces (`from_formal_adjoint`)

In a simple `EuclideanSpace`, the dual and adjoint are effectively the same. This is not true for more complex spaces, like a `Sobolev` space, which has a non-trivial inner product.

A common task is to define an operator in terms of its simple action on the underlying L² space of functions, and then "lift" it to be a proper operator on the `Sobolev` space with the correct adjoint. The `from_formal_adjoint` static
method is designed for exactly this purpose. Here we illustrate this approach using the derivative operator for functions on the unit sphere. This operator maps a Sobolev space with order $s$ into one with order $s-1$. It is not 
a well-defined operator from $L^{2}(\mathbb{S}^{1})$ to itself. But it's **formal adjoint** can be defined relative to the $L^{2}(\mathbb{S}^{1})$ inner-product by resticting attention to smooth functions, and in this manner we 
see that the operators formal adjoint is equal to the negative of itself. The following code implements this idea practically. 

In [ ]:
# Set the domain and codomain
order = 2
scale = 0.1
domain = Sobolev.from_sobolev_parameters(order, scale)
codomain = Sobolev(domain.kmax, order-1, scale)

# Get the underlying Lebesgue space on which the Sobolev spaces are built
lebesgue = domain.underlying_space

# Define the action of the derivative operator in the Fourier domain
def mapping(u):
    coeff = lebesgue.to_coefficients(u)
    k = np.arange(coeff.size)
    diff_coeff = 1j * k * coeff
    return lebesgue.from_coefficients(diff_coeff)

# Set the formal L2 operator
A_L2 = inf.LinearOperator(lebesgue, lebesgue, mapping, adjoint_mapping=lambda u : -mapping(u))

# Set the full operator using the static method. 
A = inf.LinearOperator.from_formal_adjoint(domain, codomain, A_L2)

# Run checks on the operator. 
A.check()


# Define a function on which the operator can act
u = domain.project_function(lambda th : np.sin(th))

# get the result of the derivative operator
v = A(u)
v_exact = codomain.project_function(lambda th : np.cos(th))

# Plot the results and compare to the expected function. 
fig, ax = domain.plot(u, label='input')
codomain.plot(v, fig=fig, ax=ax, label='output')
codomain.plot(v_exact, fig=fig, ax=ax, linestyle='--', label='exact')
ax.legend()
ax.set_title("Operator Test")
plt.show()


# We can also check that the adjoint identity holds 
u = domain.project_function(lambda th : np.sin(2*th))
v = codomain.project_function(lambda th : np.exp(-(th-np.pi)**2))

lhs = codomain.inner_product(A(u), v)
rhs = domain.inner_product(u, A.adjoint(v))
print(f'<Au,v>  = {lhs}')
print(f'<v,A*u> = {rhs}')
print(f'Relative difference = {np.abs(lhs-rhs)/np.abs(lhs)}')

The derivative operator for functions on a circle arises quite often within applications, and so it is implemented as a property of the `Sobolev`  class. This is shown below where we compare our implementation against that in the library.

In [ ]:
# Get the derivative operator and repeat the above tests. 
A = domain.derivative_operator

# Define a function on which the operator can act
u = domain.project_function(lambda th : np.sin(th))

# get the result of the derivative operator
v = A(u)
v_exact = codomain.project_function(lambda th : np.cos(th))

# Plot the results and compare to the expected function. 
fig, ax = domain.plot(u, label='input')
codomain.plot(v, fig=fig, ax=ax, label='output')
codomain.plot(v_exact, fig=fig, ax=ax, linestyle='--', label='exact')
ax.legend()
ax.set_title("Operator Test")
plt.show()


# We can also check that the adjoint identity holds 
u = domain.project_function(lambda th : np.sin(2*th))
v = codomain.project_function(lambda th : np.exp(-(th-np.pi)**2))

lhs = codomain.inner_product(A(u), v)
rhs = domain.inner_product(u, A.adjoint(v))
print(f'<Au,v>  = {lhs}')
print(f'<v,A*u> = {rhs}')
print(f'Relative difference = {np.abs(lhs-rhs)/np.abs(lhs)}')